<a href="https://colab.research.google.com/github/polevev/kaggle/blob/main/%D0%A0%D0%BE%D0%B6%D0%B4%D0%B0%D0%B5%D0%BC%D0%BE%D1%81%D1%82%D1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_csv('/content/rf_birthrate_dataset_full.csv')
df.sample(5)

In [ ]:
df.describe()

In [ ]:
from sklearn.preprocessing import LabelEncoder

scaler = LabelEncoder()

df['Регион'] = scaler.fit_transform(df['Регион'])
df['Год'] = scaler.fit_transform(df['Год'])

In [ ]:
df

In [ ]:
df_1 = pd.DataFrame(df[df['Регион']==88],columns=df.columns)

In [ ]:
import matplotlib.pyplot as plt

fig,axes = plt.subplots(3, 5)
fig.set_size_inches(20, 8)
i = 0
j = 0
for column in df.columns:
  axes[i,j].bar(df_1['Год'], df_1[column])
  axes[i,j].set_ylabel(column)
  j+=1
  if j%5==0:
    i+=1
    j=0
plt.show()

In [ ]:
df['Браки/разводы'] = df['Браки']/df['Разводы']

In [ ]:
df.drop(columns=['Браки','Разводы'], inplace=True)

In [ ]:
df

In [ ]:
df = df.sort_values('Год').dropna()
df['lag_1'] = df['Успешность_онлайн_знакомств'].shift(1)
df = df.dropna()

In [ ]:
df

In [ ]:
df['rolling_mean'] = df['Успешность_онлайн_знакомств'].rolling(window=3).mean()
df['year_sin'] = np.sin(2 * np.pi * df['Год'] / 10)

In [ ]:
df

In [ ]:
df = df.dropna()

In [ ]:
import seaborn as sns
sns.heatmap(df.corr(numeric_only=True))

In [ ]:
df_origin = df.copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

Scaler =  StandardScaler()

df_origin['lag_1'] = Scaler.fit_transform(df_origin[['lag_1']])

In [ ]:
df_origin = pd.DataFrame(df_origin, columns = df.columns)

In [ ]:
df_origin

In [ ]:
X = df.drop(columns=['Успешность_онлайн_знакомств'])
y = df['Успешность_онлайн_знакомств']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2,stratify=y, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb

models = {
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42),
    'LinearRegression': LinearRegression(),
    'SVR': SVR(),
    'KNeighbors': KNeighborsRegressor(),
    'xgb': xgb.XGBRegressor(random_state=42)
}

In [ ]:
from sklearn.model_selection import cross_validate, TimeSeriesSplit
import numpy as np

results = {}
for name, model in models.items():
    # Используем TimeSeriesSplit и правильные ключи метрик
    tscv = TimeSeriesSplit(n_splits=5)
    scores = cross_validate(
        model,
        X_train,
        y_train,
        cv=tscv,
        scoring=['neg_mean_squared_error', 'r2']
    )
    results[name] = {
        'MSE': -np.mean(scores['test_neg_mean_squared_error']),
        'R²': np.mean(scores['test_r2'])
    }

results_df = pd.DataFrame.from_dict(results, orient='index')
results_df

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Определяем модель
rf_model = RandomForestRegressor(random_state=42)

# Определяем параметры для поиска
param_grid = {
    'n_estimators': [100, 200, 300, 500],  # Количество деревьев
    'max_depth': [10, 20, 30],  # Максимальная глубина деревьев
    'min_samples_split': [2, 5, 10],  # Минимальное количество сэмплов для разбиения узла
    'min_samples_leaf': [2, 4, 6],  # Минимальное количество сэмплов в листе
    'max_features': ['auto', 'sqrt', 'log2'],  # Количество признаков для поиска лучшего разбиения
}

# Инициализация GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, scoring='neg_mean_squared_error', verbose=True)

# Обучение на данных
grid_search.fit(X_train, y_train)

# Вывод лучших параметров
print("Best parameters found: ", grid_search.best_params_)

# Оценка модели с лучшими гиперпараметрами
best_model = grid_search.best_estimator_
best_score = grid_search.best_score_

print(f"Best score (MSE): {best_score:.4f}")

In [ ]:
scoring = ['neg_mean_squared_error', 'r2']
tscv = TimeSeriesSplit(n_splits=5)
scores = cross_validate(best_model, X_train, y_train, cv=tscv, scoring=scoring)
scores

In [ ]:
y_pred = best_model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
print(f'MAE:{mean_absolute_error(y_test, y_pred)}')
print(f'MSE:{mean_squared_error(y_test, y_pred)}')
print(f'R^2:{r2_score(y_test, y_pred)}')

In [ ]:
import torch

In [ ]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(num_classes, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Linear(32,1)
        )


    def forward(self, x):
      return (self.fc(x))

In [ ]:
X_train_tensor = torch.from_numpy(X_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [ ]:
train_loader = torch.utils.data.DataLoader(list(zip(X_train_tensor, y_train_tensor)), batch_size=64, shuffle=True)

In [ ]:
X_train.shape

In [ ]:
model = Net(num_classes=16)

criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import tqdm
from tqdm.auto import tqdm

def train_stochastic(model, loader, criterion, optimizer, num_epoch):
    # цикл по количеству эпох обучения
    total_epoch_loss = []
    for t in tqdm(range(num_epoch)):
        epoch_loss = []

        # цикл по батчам даталоадера
        for X_batch, y_batch in loader:
            # Вычислим предсказания нашей модели
            y_batch = y_batch.unsqueeze(1)
            y_pred = model(X_batch)

            # Посчитаем значение функции потерь  на полученном предсказании
            loss = criterion(y_pred, y_batch)
            epoch_loss.append(loss.item())

            # Выполним подсчёт новых градиентов
            loss.backward()
            # Выполним шаг градиентного спуска
            optimizer.step()
            # Обнулим сохраненные у оптимизатора значения градиентов
            # перед следующим шагом обучения
            optimizer.zero_grad()

        print(f"{t} epoch: curr_loss = {np.mean(epoch_loss)}")
        total_epoch_loss.append(epoch_loss)

    plt.figure(figsize=(10,8))
    plt.plot(range(1, num_epoch+1), total_epoch_loss, 'b-', linewidth=2)
    plt.title('Training Loss Over Epochs', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.grid(True)
    plt.show()

    return model

In [ ]:
model = train_stochastic(model, train_loader, criterion, optimizer, 500)

In [ ]:
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

In [ ]:
with torch.no_grad():
    nn_prediction = model(torch.FloatTensor(X_test_tensor))
    nn_prediction = nn_prediction.tolist()

In [ ]:
print(f'MAE:{mean_absolute_error(y_test, nn_prediction)}')
print(f'MSE:{mean_squared_error(y_test, nn_prediction)}')
print(f'R^2:{r2_score(y_test, nn_prediction)}')